## Data Loading and Cleaning

### Table of Contents
- Introduction
- Import and Load Raw Data
- Data Cleaning
- Conclusion
- Export Files

### Introduction

This notebook is to gather the input data, clean the data, and export a dataset for exploratory data analysis (EDA). The initial dataset was downloaded from the Bureau of Transportation Statistics website, specifically the database of international direct flights with arrival to or departure from United States. A selection of fields were available and the fields that I thought would be useful for this project were chosen. At the time of starting this project, records from January 1990 to May 2024 were available, however, for this project I decided to evaluate the last decade from Jan 2013 to May 2024. Let's load in the data and get to cleaning.

### Import and Load Raw Data

First things first, import the necessary libraries.

In [1]:
# import libraries
import numpy as np
import pandas as pd
import glob

Since each year of the database were downloaded as an individual file, we will use the `glob` function to compile the files into one variable.

In [2]:
# compile the data files
file_paths = glob.glob('../data/20*')
file_paths

['../data\\2013.csv',
 '../data\\2014.csv',
 '../data\\2015.csv',
 '../data\\2016.csv',
 '../data\\2017.csv',
 '../data\\2018.csv',
 '../data\\2019.csv',
 '../data\\2020.csv',
 '../data\\2021.csv',
 '../data\\2022.csv',
 '../data\\2023.csv',
 '../data\\2024.csv']

Now that the files are loaded into one variable, it can be easily read in using the function `pd.read_cvs` with a for loop and then converted into a pandas dataframe.

In [3]:
# combine data files into one dataframe
df = pd.DataFrame()

# loop thru the csv files to combine them into one dataframe
for file in file_paths:
    df_air = pd.read_csv(file, index_col=0)
    df = pd.concat([df,df_air])

df = df.reset_index()

Let's take a look at the newly created dataframe.

In [10]:
# check the first few rows
df.head()

,DEPARTURES_PERFORMED,SEATS,PASSENGERS,CARRIER,CARRIER_NAME,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,ORIGIN_COUNTRY_NAME,DEST_AIRPORT_ID,DEST_CITY_NAME,DEST_COUNTRY_NAME,YEAR,MONTH,DATE
0,0.0,0.0,0.0,3M,Silver Airways,11624,"Key West, FL",United States,11534,"North Eleuthera, The Bahamas",The Bahamas,2013,10,2013-10-01
1,0.0,0.0,0.0,3M,Silver Airways,11624,"Key West, FL",United States,11534,"North Eleuthera, The Bahamas",The Bahamas,2013,11,2013-11-01
2,0.0,0.0,0.0,3M,Silver Airways,13289,"Marsh Harbour, The Bahamas",The Bahamas,15304,"Tampa, FL",United States,2013,2,2013-02-01
3,0.0,0.0,0.0,3M,Silver Airways,15304,"Tampa, FL",United States,11534,"North Eleuthera, The Bahamas",The Bahamas,2013,2,2013-02-01
4,0.0,0.0,0.0,3M,Silver Airways,15304,"Tampa, FL",United States,11534,"North Eleuthera, The Bahamas",The Bahamas,2013,10,2013-10-01


In [4]:
# an informational look at the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978570 entries, 0 to 978569
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   DEPARTURES_PERFORMED  978570 non-null  float64
 1   SEATS                 978570 non-null  float64
 2   PASSENGERS            978570 non-null  float64
 3   CARRIER               978478 non-null  object 
 4   CARRIER_NAME          978395 non-null  object 
 5   ORIGIN_AIRPORT_ID     978570 non-null  int64  
 6   ORIGIN_CITY_NAME      978169 non-null  object 
 7   ORIGIN_COUNTRY_NAME   978169 non-null  object 
 8   DEST_AIRPORT_ID       978570 non-null  int64  
 9   DEST_CITY_NAME        978170 non-null  object 
 10  DEST_COUNTRY_NAME     978170 non-null  object 
 11  YEAR                  978570 non-null  int64  
 12  MONTH                 978570 non-null  int64  
dtypes: float64(3), int64(4), object(6)
memory usage: 97.1+ MB


There are 978,570 entries and it looks like some of the columns such as `CARRIER_NAME` are missing data. Also, the dataframe seems to be not sorted by date. We will deal with the missing values and order, but first let's take a look at the data types. <br>
As expected, there's a mix of number and object (text) data types in the dataframe. As this will be a time series model, the `YEAR` and `MONTH` columns can be combined to make a new `DATE` column as datetime data type.

In [5]:
# convert to datetime datatype
df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH']].assign(DAY=1))

In [6]:
# check that the DATE column is created
df.head()

,DEPARTURES_PERFORMED,SEATS,PASSENGERS,CARRIER,CARRIER_NAME,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,ORIGIN_COUNTRY_NAME,DEST_AIRPORT_ID,DEST_CITY_NAME,DEST_COUNTRY_NAME,YEAR,MONTH,DATE
0,0.0,0.0,0.0,3M,Silver Airways,11624,"Key West, FL",United States,11534,"North Eleuthera, The Bahamas",The Bahamas,2013,10,2013-10-01
1,0.0,0.0,0.0,3M,Silver Airways,11624,"Key West, FL",United States,11534,"North Eleuthera, The Bahamas",The Bahamas,2013,11,2013-11-01
2,0.0,0.0,0.0,3M,Silver Airways,13289,"Marsh Harbour, The Bahamas",The Bahamas,15304,"Tampa, FL",United States,2013,2,2013-02-01
3,0.0,0.0,0.0,3M,Silver Airways,15304,"Tampa, FL",United States,11534,"North Eleuthera, The Bahamas",The Bahamas,2013,2,2013-02-01
4,0.0,0.0,0.0,3M,Silver Airways,15304,"Tampa, FL",United States,11534,"North Eleuthera, The Bahamas",The Bahamas,2013,10,2013-10-01


In [7]:
# check the data type of the DATE column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978570 entries, 0 to 978569
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   DEPARTURES_PERFORMED  978570 non-null  float64       
 1   SEATS                 978570 non-null  float64       
 2   PASSENGERS            978570 non-null  float64       
 3   CARRIER               978478 non-null  object        
 4   CARRIER_NAME          978395 non-null  object        
 5   ORIGIN_AIRPORT_ID     978570 non-null  int64         
 6   ORIGIN_CITY_NAME      978169 non-null  object        
 7   ORIGIN_COUNTRY_NAME   978169 non-null  object        
 8   DEST_AIRPORT_ID       978570 non-null  int64         
 9   DEST_CITY_NAME        978170 non-null  object        
 10  DEST_COUNTRY_NAME     978170 non-null  object        
 11  YEAR                  978570 non-null  int64         
 12  MONTH                 978570 non-null  int64         
 13 

Now that the data is inspected, let's get to data cleaning.

### Data Cleaning

As noted from the initial inspection of the dataframe, we will look at the missing data.

In [16]:
# check missing data
df.isna().sum()

DEPARTURES_PERFORMED      0
SEATS                     0
PASSENGERS                0
CARRIER                  92
CARRIER_NAME            175
ORIGIN_AIRPORT_ID         0
ORIGIN_CITY_NAME        401
ORIGIN_COUNTRY_NAME     401
DEST_AIRPORT_ID           0
DEST_CITY_NAME          400
DEST_COUNTRY_NAME       400
YEAR                      0
MONTH                     0
DATE                      0
dtype: int64

In [17]:
# percentage of missing data
round(df.isna().mean()*100,3)

DEPARTURES_PERFORMED    0.000
SEATS                   0.000
PASSENGERS              0.000
CARRIER                 0.009
CARRIER_NAME            0.018
ORIGIN_AIRPORT_ID       0.000
ORIGIN_CITY_NAME        0.041
ORIGIN_COUNTRY_NAME     0.041
DEST_AIRPORT_ID         0.000
DEST_CITY_NAME          0.041
DEST_COUNTRY_NAME       0.041
YEAR                    0.000
MONTH                   0.000
DATE                    0.000
dtype: float64

Six columns have missing data and for each column a very small percentage, less than 0.050%, is impacted. The missing values (nulls) could be disregared by simply dropping the rows with them, or we can take a look at them and see if they can be easily filled in.<br>
We can start with the columns with the biggest percentage since they have equal amount of data missing and so they could be correlated: `ORIGIN_CITY_NAME`, `ORIGIN_COUNTRY_NAME`, `DEST_CITY_NAME`, `DEST_COUNTRY_NAME`

In [10]:
# check unique null values
df['ORIGIN_AIRPORT_ID'][df['ORIGIN_CITY_NAME'].isna()].value_counts()

ORIGIN_AIRPORT_ID
11369    401
Name: count, dtype: int64

In [11]:
# determined from the lookup table 'L_AIRPORT_ID'
df.loc[df['ORIGIN_AIRPORT_ID'] == 11369, ['ORIGIN_CITY_NAME','ORIGIN_COUNTRY_NAME']] = ['Doha','Qatar']

We filter for the rows where `ORIGIN_CITY_NAME` are null and since `ORIGIN_AIRPORT_ID` does not have any nulls, we can extract the airport ID for reference. All the null values are associated with airport ID 11369. Using the lookup table 'L_AIRPORT_ID' for reference which can be found in the 'references' folder, airport ID 11369 is Doha, Qatar. The number of nulls for `ORIGIN_CITY_NAME` and `ORIGIN_COUNTRY_NAME` are equal so we can assume Qatar is the missing country name.

In [12]:
# check unique null values
df['DEST_AIRPORT_ID'][df['DEST_CITY_NAME'].isna()].value_counts()

DEST_AIRPORT_ID
11369    400
Name: count, dtype: int64

In [13]:
# determined from the lookup table 'L_AIRPORT_ID'
df.loc[df['DEST_AIRPORT_ID'] == 11369, ['DEST_CITY_NAME','DEST_COUNTRY_NAME']] = ['Doha','Qatar']

Same approach is taken for `DEST_CITY_NAME` and `DEST_COUNTRY_NAME` with `DEST_AIRPORT_ID` for reference. Once again Doha, Qatar was the missing piece.

In [14]:
# check unique null values
df['CARRIER'][df['CARRIER_NAME'].isna()].value_counts()

CARRIER
K8     167
2HQ      8
Name: count, dtype: int64

In [15]:
# determined from the lookup table 'L_CARRIER_HISTORY'
df.loc[df['CARRIER'] == 'K8', ['CARRIER_NAME']] = ['Caribbean Sun Airlines, Inc. d/b/a World Atlantic Airlines']

In [16]:
# determined from the lookup table 'L_CARRIER_HISTORY'
df.loc[df['CARRIER'] == '2HQ', ['CARRIER_NAME']] = ['Elite Airways LLC']

Similar approach is taken for `CARRIER_NAME` where `CARRIER` is used as reference and lookup table 'L_CARRIER_HISTORY' is used.

In [17]:
df['CARRIER_NAME'][df['CARRIER'].isna()].value_counts()

CARRIER_NAME
North American Airlines    92
Name: count, dtype: int64

In [18]:
# determined from the lookup table 'L_CARRIER_HISTORY'
df.loc[df['CARRIER_NAME'] == 'North American Airlines', ['CARRIER']] = ['NA']

Inverse approach is taken for `CARRIER` where this time `CARRIER_NAME` is used as reference and lookup table 'L_CARRIER_HISTORY' is used.

In [19]:
df.isna().sum()

DEPARTURES_PERFORMED    0
SEATS                   0
PASSENGERS              0
CARRIER                 0
CARRIER_NAME            0
ORIGIN_AIRPORT_ID       0
ORIGIN_CITY_NAME        0
ORIGIN_COUNTRY_NAME     0
DEST_AIRPORT_ID         0
DEST_CITY_NAME          0
DEST_COUNTRY_NAME       0
YEAR                    0
MONTH                   0
DATE                    0
dtype: int64

Now checking for null values, we can see all the null values are taken care of and also that our assumption about city and country was true.

To clean up the dataset for EDA, it needs to be filtered and narrowed down according to the goal of this project. The ultimate goal is to forecast popular destinations outside of United States. The dataset is records of all scheduled non-stop flights in and out of United States. As there may have been scheduled flights that never happened or rescheduled due to some cancellation reasons, the dataset is filtered for completed commercial passenger flights. Also it is filtered for flights (travel) outbound from USA to international destinations.

In [20]:
# destinations outside of USA and completed commercial passenger flight
df_air = df[(df['DEST_COUNTRY_NAME'] != 'United States') & (df['DEPARTURES_PERFORMED'] > 0) & (df['SEATS'] > 0) & (df['PASSENGERS'] > 0)]
df_air.head()

,DEPARTURES_PERFORMED,SEATS,PASSENGERS,CARRIER,CARRIER_NAME,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,ORIGIN_COUNTRY_NAME,DEST_AIRPORT_ID,DEST_CITY_NAME,DEST_COUNTRY_NAME,YEAR,MONTH,DATE
3727,1.0,1.0,1.0,12Q,Unijet,14679,"San Diego, CA",United States,12900,"Paris, France",France,2013,4,2013-04-01
3735,1.0,2.0,2.0,KAH,Kenmore Air Harbor,12652,"Kenmore, WA",United States,16263,"Victoria, Canada",Canada,2013,10,2013-10-01
3736,1.0,2.0,2.0,KAH,Kenmore Air Harbor,12652,"Kenmore, WA",United States,16263,"Victoria, Canada",Canada,2013,11,2013-11-01
3737,1.0,4.0,1.0,04Q,Tradewind Aviation,12197,"White Plains, NY",United States,16215,"Toronto, Canada",Canada,2013,2,2013-02-01
3739,1.0,4.0,2.0,KAH,Kenmore Air Harbor,13002,"Seattle, WA",United States,16263,"Victoria, Canada",Canada,2013,5,2013-05-01


In [21]:
df_air['DEST_COUNTRY_NAME'].value_counts()

DEST_COUNTRY_NAME
Canada                74976
Mexico                74043
United Kingdom        19820
Dominican Republic    14347
The Bahamas           12765
                      ...  
Tanzania                  1
Laos                      1
Macedonia                 1
Armenia                   1
Mauritania                1
Name: count, Length: 171, dtype: int64

Looking at the unique count of destinations outside of United States, there were flights to 171 different destinations from Jan 2013 to May 2024. <br>

Final part of the data cleanup is taking care of duplicate data.

In [22]:
# check for duplicate data
df_air.duplicated().sum()

39

In [23]:
# view the duplicate data
df_air[df_air.duplicated(keep=False)]

,DEPARTURES_PERFORMED,SEATS,PASSENGERS,CARRIER,CARRIER_NAME,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,ORIGIN_COUNTRY_NAME,DEST_AIRPORT_ID,DEST_CITY_NAME,DEST_COUNTRY_NAME,YEAR,MONTH,DATE
79301,1.0,2.0,2.0,KAH,Kenmore Air Harbor,13002,"Seattle, WA",United States,16310,"Nanaimo, Canada",Canada,2014,9,2014-09-01
79302,1.0,2.0,2.0,KAH,Kenmore Air Harbor,13002,"Seattle, WA",United States,16310,"Nanaimo, Canada",Canada,2014,9,2014-09-01
158385,1.0,2.0,2.0,KAH,Kenmore Air Harbor,13002,"Seattle, WA",United States,16233,"Victoria, Canada",Canada,2015,3,2015-03-01
158386,1.0,2.0,2.0,KAH,Kenmore Air Harbor,13002,"Seattle, WA",United States,16233,"Victoria, Canada",Canada,2015,3,2015-03-01
158497,1.0,6.0,4.0,KAH,Kenmore Air Harbor,12652,"Kenmore, WA",United States,16233,"Victoria, Canada",Canada,2015,8,2015-08-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873869,4.0,972.0,864.0,LA,Lan-Chile Airlines,12478,"New York, NY",United States,12986,"Lima, Peru",Peru,2023,10,2023-10-01
919918,31.0,5332.0,4495.0,AA,American Airlines Inc.,13303,"Miami, FL",United States,10411,"Aruba, Aruba",Aruba,2023,3,2023-03-01
919919,31.0,5332.0,4495.0,AA,American Airlines Inc.,13303,"Miami, FL",United States,10411,"Aruba, Aruba",Aruba,2023,3,2023-03-01
921591,31.0,8184.0,7169.0,SQ,Singapore Airlines Ltd.,12892,"Los Angeles, CA",United States,13744,"Tokyo, Japan",Japan,2023,3,2023-03-01


There are 39 duplicated data and looking through them they can simply be dropped.

In [24]:
# drop duplicate data and verify
df_air = df_air.drop_duplicates()
df_air.duplicated().sum()

0

Now that null values and duplicate data are dealt with and the database is modified in tune for EDA, last thing to do is sort the dataframe by date.

In [25]:
# sort date ascending
df_air.sort_values(by='DATE', inplace=True)
# reset the index after the sort
df_air.reset_index(drop=True, inplace=True)

In [30]:
# verify sort and index reset
print(df_air.info())
df_air.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390469 entries, 0 to 390468
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   DEPARTURES_PERFORMED  390469 non-null  float64       
 1   SEATS                 390469 non-null  float64       
 2   PASSENGERS            390469 non-null  float64       
 3   CARRIER               390469 non-null  object        
 4   CARRIER_NAME          390469 non-null  object        
 5   ORIGIN_AIRPORT_ID     390469 non-null  int64         
 6   ORIGIN_CITY_NAME      390469 non-null  object        
 7   ORIGIN_COUNTRY_NAME   390469 non-null  object        
 8   DEST_AIRPORT_ID       390469 non-null  int64         
 9   DEST_CITY_NAME        390469 non-null  object        
 10  DEST_COUNTRY_NAME     390469 non-null  object        
 11  YEAR                  390469 non-null  int64         
 12  MONTH                 390469 non-null  int64         
 13 

,DEPARTURES_PERFORMED,SEATS,PASSENGERS,CARRIER,CARRIER_NAME,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,ORIGIN_COUNTRY_NAME,DEST_AIRPORT_ID,DEST_CITY_NAME,DEST_COUNTRY_NAME,YEAR,MONTH,DATE
390464,4.0,744.0,604.0,G4,Allegiant Air,14122,"Pittsburgh, PA",United States,11032,"Cancun, Mexico",Mexico,2024,5,2024-05-01
390465,30.0,10500.0,5687.0,UA,United Air Lines Inc.,14771,"San Francisco, CA",United States,15092,"Sydney, Australia",Australia,2024,5,2024-05-01
390466,4.0,744.0,727.0,SY,Sun Country Airlines d/b/a MN Airlines,14683,"San Antonio, TX",United States,11032,"Cancun, Mexico",Mexico,2024,5,2024-05-01
390467,30.0,10500.0,9405.0,UA,United Air Lines Inc.,12264,"Washington, DC",United States,11760,"Frankfurt, Germany",Germany,2024,5,2024-05-01
390468,4.0,636.0,563.0,B6,JetBlue Airways,12478,"New York, NY",United States,12990,"Liberia, Costa Rica",Costa Rica,2024,5,2024-05-01


### Conclusion
Dataframe is sorted by date, contains no null or duplicated data, and ready for EDA. The cleaned dataset can be exported.

### Export Files

In [28]:
# export cleaned dataset
df_air.to_csv('capstone_data_cleaned.csv', index=False) # makes it not save an index col